In [202]:
#imports
import mne
import glob
import os
import re
import numpy as np
from itertools import chain
from matplotlib import pyplot as plt
from mne.time_frequency import psd_multitaper
from mne.decoding import LinearModel
from mne.decoding import get_coef
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import pandas as pd

In [203]:
# path to data 
path = '/Users/ilamiheev/Downloads/eeg_data'
# create folders for results
path_res = '/Users/ilamiheev/Downloads/results_ihna'
path_subj = os.path.join(path_res, 'subjects_classification')
path_subj_topo = os.path.join(path_subj, 'subjects_topo')
path_unite_subj = os.path.join(path_res, 'unite_subjects')
path_unite_topo =  os.path.join(path_unite_subj, 'unite_topo')
path_group = os.path.join(path_res, 'group_classification')
path_group_topo =  os.path.join(path_group, 'group_topo')
for pathn in [path_res,path_subj,path_subj_topo,path_unite_subj,path_unite_topo,path_group,path_group_topo]:
    os.makedirs(pathn)

PermissionError: [Errno 13] Permission denied: '/Users/ilamiheev'

In [ ]:
path = '/Users/ilyamikheev/Documents/IHNA/eeg_data'

In [ ]:

#Create progress bar for class bar
from tqdm.notebook import tqdm

In [ ]:
class Data:
    """
    class for dataset creation
    return dictionary 
    """
    def __init__(self, files, indexes, path, drop_chan, classes, fr_bands):
        montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
        epochs_list_fin = []
        paths = glob.glob(path + '/{0}/Reals/*.edf'.format(file_name))
        epochs_list = self._create_epochs(classes, paths)
        for teta in range(len(epochs_list)):
            new_names = dict(
                    (ch_name,
                     ch_name.replace('-', '').replace('Chan ', 'E').replace('CAR', '').replace('EEG ', '')
                     .replace('CA', '').replace(' ', ''))
                    for ch_name in epochs_list[teta].ch_names)
                epochs_list[teta].rename_channels(new_names).set_montage(montage).drop_channels(drop_chan)
            #self.__setitem__ (ind, epochs_list[0].get_data)
        self.data = epochs_list[0].get_data
        self.chan_names = epochs_list[0].ch_names
        self.fr_bands
    def __getitem__(self, key):
        return self.__dict__[key]
    def __setitem__(self, key, item):
        self.__dict__[key] = item
    def __iter__(self):
        return iter(self.__dict__)

    @staticmethod
    def _create_epochs(events_list, paths):
        epochs_list = []
        for event_ind in events_list:
            j = 0
            for i in [x for x in paths if '{0}'.format(event_ind) in x]:
                event_id = dict(a=event_ind)
                raw = mne.io.read_raw_edf(i, preload=True, verbose='ERROR')
                raw.filter(3, 25., fir_design='firwin')
                if len(raw.times) // 500 < 10:
                    continue
                new_events = mne.make_fixed_length_events(raw, id=event_ind, start=5, duration=2)
                if j == 1:
                    epochs = mne.concatenate_epochs([mne.Epochs(raw, new_events, event_id=event_id, tmin=0,
                                                                tmax=1.99, baseline=None, flat=dict(eeg=1e-20),
                                                                preload=True, verbose=False), epochs])
                else:
                    epochs = mne.Epochs(raw, new_events, event_id=event_id, tmin=0, tmax=1.99, baseline=None,
                                        flat=dict(eeg=1e-20), preload=True, verbose=False)
                    j += 1
            epochs_list.append(epochs)
        return epochs_list

    def eeg_power_band(self, key):
        fr_bands = self.fr_bands
        fin_table, fin_feat = [], []
        for beta in range(len(self[key])):
            psds, freqs = psd_multitaper(self[key][beta])
            psds_table = np.mean(psds, axis=0)
            psds /= psds.sum(axis=-1)[..., None]
            psds_table /= psds_table.sum(axis=-1)[..., None]
            psd_table_list, psd_features_list = [], []
            for fmin, fmax in fr_bands.values():
                freq_mask = (fmin < freqs) & (freqs < fmax)
                data_table, data_feat = psds_table[..., freq_mask].mean(axis=-1), psds[..., freq_mask].mean(axis=-1)
                psd_features_list.append(data_feat)
                psd_table_list.append(data_table)
            fin_table.append(psd_table_list)
            fin_feat.append(psd_features_list)
        return fin_feat, fin_table

In [ ]:
path = '/Users/ilamiheev/Downloads/eeg_data'
# create folders for results
path_res = '/Users/ilamiheev/Downloads/results_ihna'
path_subj = os.path.join(path_res, 'subjects_classification')
path_subj_topo = os.path.join(path_subj, 'subjects_topo')
path_unite_subj = os.path.join(path_res, 'unite_subjects')
path_unite_topo =  os.path.join(path_unite_subj, 'unite_topo')
path_group = os.path.join(path_res, 'group_classification')
path_group_topo =  os.path.join(path_group, 'group_topo')

In [ ]:
data = Data(files, indexes, path, drop_chan = chan_drop, classes = events_list, fr_bands = fr_bands)

In [ ]:
epochs_list_fin = []
        for file_name, ind in (zip(tqdm(files),indexes)):
            paths = glob.glob(path + '/{0}/Reals/*.edf'.format(file_name))
            epochs_list = self._create_epochs(classes, paths)
            for teta in range(len(epochs_list)):
                new_names = dict(
                    (ch_name,
                     ch_name.replace('-', '').replace('Chan ', 'E').replace('CAR', '').replace('EEG ', '')
                     .replace('CA', '').replace(' ', ''))
                    for ch_name in epochs_list[teta].ch_names)
                epochs_list[teta].rename_channels(new_names).set_montage(montage).drop_channels(drop_chan)
            self.__setitem__ (ind, epochs_list[0].get_data)
        self.chan_names = epochs_list[0].ch_names
        self.fr_bands

In [ ]:
epochs_list

In [ ]:
# eeg parameters and subjects indexes
files = [f for f in sorted(os.listdir(path))]
_ = files.pop(0)
indexes = []
for i,j in enumerate(files):
    indexes.append(re.search('(.+?)_', j).group(1))
chan_drop = ['E8','E14','E21','E25','E43','E48','E49','E56','E57','E63','E64','E65','E68','E69','E73','E74','E81'
             ,'E82','E88','E89','E90','E94','E95','E99','E100','E107','E113','E119','E120','E125','E126','E127'
             ,'E128','Status']
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
events_list = [241,242,244]
fr_bands = {   "theta1":  [4,6],
               "theta2":  [6,8],
               "alpha1": [8,10],
               "alpha2": [10,12],
               "beta1":  [12,16],
               "beta2":  [16,20],
               "beta3":  [20,24] }
dict_cls = { "241/244": [0,2],
             "242/244": [1,2],
             "241/242": [0,1] }
mat = ['311','312','314','315','316','317','326','327','328','330','334','335']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:
import numpy as np

In [ ]:
#samples/features 1/features
def predict_subject(x, y, Pattern):
    prec_x = np.linalg.pinv(np.cov(x.T))
    new_filter = prec_x.dot(Pattern.T).T
    scores = np.dot(X, Filter)
    scores.ravel()
    indices = (scores > 0).astype(int)
    cl = np.array([0,1])
    y_predict = cl[indices]
    a_s = accuracy_score(y, y_predict)
    return s_cl

In [ ]:
cl = np.array([0,1])
ind = np.array([0,1,1,0])
cl[ind]

In [ ]:
indices

In [ ]:
# estimate relative power 
def eeg_power_band(epochs_list):
    fin_table, fin_feat = [], []
    for beta in range(len(epochs_list)):
        psds, freqs = psd_multitaper(epochs_list[beta])
        psds_table = np.mean(psds, axis=0)
        psds /= psds.sum(axis=-1)[..., None]
        psds_table /= psds_table.sum(axis=-1)[..., None]
        psd_table_list, psd_features_list = [], []
        for fmin, fmax in fr_bands.values():
            freq_mask = (fmin < freqs) & (freqs < fmax)
            data_table, data_feat = psds_table[..., freq_mask].mean(axis=-1), psds[..., freq_mask].mean(axis=-1)
            psd_features_list.append(data_feat)
            psd_table_list.append(data_table)
        fin_table.append(psd_table_list)
        fin_feat.append(psd_features_list)
    return fin_feat, fin_table
# logistic regression with l2 penalty and CV
# for cv ev roc/auc score 
def predict(x_train, x_test, y_train, y_test):
    results = np.zeros((1,4))
    model = make_pipeline( StandardScaler(),                  
                               LinearModel(LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='roc_auc',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10 
                                                                       ))) 
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    ac = accuracy_score(y_test,y_predict)
    score = model.score(x_test,y_test)
    #cm=confusion_matrix(y_test,y_predict)
    #TN,TP,FN,FP = cm[1,1],cm[0,0],cm[1,0],cm[0,1]
    #results[0,0], results[0,1] = accuracy_score(y_test,y_predict), 0
    #results[0,2], results[0,3] = TP/float(TP+FN), TN/float(TN+FP)
    
    return ac, score, model
# plot topomap for each freq band
def plot_topo(ar):
    vmin = np.amin(ar)
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(30, 20)) 
    for name, pos, plot_name, ind in zip(('patterns_', 'filters_'),(0.8,0.5),
                                         ('Patterns','Filters'),(0,1)):
        for i,key in enumerate(list(fr_bands.keys())):
            a = mne.viz.plot_topomap(ar[ind,i,:],info,vmin=vmin,vmax=vmax, axes=axes[ind,i], 
                                 show = False)
            axes[ind,i].set(title='{}-{} Hz'.format(*fr_bands[key]))
            mne.viz.tight_layout() 
        plt.figtext(0.5,pos,'{}'.format(plot_name), va="center", ha="center", size=24, fontweight = 'semibold',)
    return fig

In [ ]:
# create features
subj_list_table, subj_list_features, e_list = [], [], []
for file_name in files:
    paths = glob.glob(path + '/{0}/Reals/*.edf'.format(file_name))
    epochs_list = []
    for event_ind in events_list:
        j = 0
        for i in [x for x in paths if '{0}'.format(event_ind) in x]: 
            event_id = dict(a=event_ind)
            raw = mne.io.read_raw_edf(i)
            if  len(raw.times)//500 < 10:
                continue  
            new_events = mne.make_fixed_length_events(raw, id=event_ind, start=5, duration=2, overlap=0)
            if j==1:
                epochs = mne.concatenate_epochs([mne.Epochs(raw, new_events, event_id = event_id, tmin=0, 
                                                            tmax=2, baseline=None, flat=dict(eeg=1e-20), 
                                                            preload=True), epochs])
            else:
                epochs = mne.Epochs(raw, new_events, event_id = event_id , tmin=0, tmax=2, baseline=None, 
                                    flat=dict(eeg=1e-20), preload=True)
                j+=1
        epochs_list.append(epochs.copy())
    for teta in range(len(epochs_list)):
            new_names = {}
            new_names = dict(
                    (ch_name,
                     ch_name.replace('-', '').replace('Chan ', 'E').replace('CAR', '').replace('EEG ', '')
                     .replace('CA', '').replace(' ', ''))
                     for ch_name in epochs_list[teta].ch_names)
            epochs_list[teta].rename_channels(new_names)
            epochs_list[teta].set_montage(montage)
            epochs_list[teta].drop_channels(chan_drop)
    e_list.append(epochs_list)
    feat_list, tabl_list = eeg_power_band(epochs_list)
    subj_list_table.append(tabl_list)
    subj_list_features.append(feat_list)
chan1 = epochs_list[0].ch_names

In [ ]:
l_lens = np.zeros((26,3))

for i in range(len(indexes)):
     for k in range(3):
         l_lens[i,k] = len(e_list[i][k])

In [ ]:
e_list[0]

df_subj = pd.DataFrame(l_lens, columns=['241', '242', '244'], index=indexes)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'subjects_epochs.xlsx'))
df_subj.to_excel(writer)
writer.save()


In [223]:
# for resting state
#path = '/Users/ilyamikheev/Downloads/19_CHANNEL_split_REST_open_for_Ilia'
path = '/Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)'
chan_drop = ['E8','E14','E21','E25','E43','E48','E49','E56','E57','E63','E64','E65','E68','E69','E73','E74','E81'
             ,'E82','E88','E89','E90','E94','E95','E99','E100','E107','E113','E119','E120','E125','E126','E127'
             ,'E128']
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
events_list = [241,242,244]
fr_bands = {   "theta1":  [4,6],
               "theta2":  [6,8],
               "alpha1": [8,10],
               "alpha2": [10,12],
               "beta1":  [12,16],
               "beta2":  [16,20],
               "beta3":  [20,24] }
dict_cls = { "241/244": [0,2],
             "242/244": [1,2],
             "241/242": [0,1] }
files = [f for f in sorted(os.listdir(path))]
_ = files.pop(0)
indexes = []
for i,j in enumerate(files):
    indexes.append(re.search('(.+?)_', j).group(1))
mat = ['311','312','314','315','316','317','326','327','328','330','334','335']
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]
subj_list_table, subj_list_features, e_list = [], [], []
for file_name in files:
    paths = glob.glob(path + '/{0}/*.edf'.format(file_name))
    epochs_list = []
    j = 0
    for i in paths:
        event_id = dict(a='0')
        raw = mne.io.read_raw_edf(i)
        if 'Markers' in raw.ch_names:
           raw.drop_channels(['Markers'])
        else:
           raw.drop_channels(['Status'])
        if  len(raw.times)//250 < 10:
            continue
        new_events = mne.make_fixed_length_events(raw, id=0, start=0, duration=2, overlap=0)
        if j==1:
            epochs = mne.concatenate_epochs([mne.Epochs(raw, new_events, event_id = 0, tmin=0,
                                                        tmax=2, baseline=None, flat=dict(eeg=1e-20),
                                                        preload=True), epochs])
        else:
            epochs = mne.Epochs(raw, new_events, event_id = 0 , tmin=0, tmax=2, baseline=None,
                                flat=dict(eeg=1e-20), preload=True)
            j+=1
    epochs_list.append(epochs.copy())
    for teta in range(len(epochs_list)):
            new_names = {}
            new_names = dict(
                    (ch_name,
                     ch_name.replace('-', '').replace('Chan ', 'E').replace('CAR', '').replace('EEG ', '')
                     .replace('CA', '').replace(' ', ''))
                     for ch_name in epochs_list[teta].ch_names)
            epochs_list[teta].rename_channels(new_names)
            epochs_list[teta].set_montage(montage)
            epochs_list[teta].drop_channels(chan_drop)
    e_list.append(epochs_list)
    feat_list, tabl_list = eeg_power_band(epochs_list)
    subj_list_table.append(tabl_list)
    subj_list_features.append(feat_list)
chan1 = epochs_list[0].ch_names


Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/311_Math_rest_open/311_Math_rest_open_n_7_m_212_t_215_eda_0_.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Loading data for 8 events and 1001 original time points ...
1 bad epochs dropped
Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/311_Math_rest_open/311_Math_rest_open_n_14_m_212_t_215_eda_0_.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Not setting metadata
Not setting metadata
8 matching events found
No baseline correction applied
0 projection items activated
Loading data for 8 events and 1001 original time points ...
1 bad epochs dropped
Not setting metadata
Not setting metadata
14 ma

In [ ]:
# for resting state
#path = '/Users/ilyamikheev/Downloads/19_CHANNEL_split_REST_open_for_Ilia'
path = '/Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)'
chan_drop = ['E8','E14','E21','E25','E43','E48','E49','E56','E57','E63','E64','E65','E68','E69','E73','E74','E81'
             ,'E82','E88','E89','E90','E94','E95','E99','E100','E107','E113','E119','E120','E125','E126','E127'
             ,'E128']
montage = mne.channels.make_standard_montage('GSN-HydroCel-128')
events_list = [241,242,244]
fr_bands = {   "theta1":  [4,6],
               "theta2":  [6,8],
               "alpha1": [8,10],
               "alpha2": [10,12],
               "beta1":  [12,16],
               "beta2":  [16,20],
               "beta3":  [20,24] }
dict_cls = { "241/244": [0,2],
             "242/244": [1,2],
             "241/242": [0,1] }
files = [f for f in sorted(os.listdir(path))]
_ = files.pop(0)
indexes = []
for i,j in enumerate(files):
    indexes.append(re.search('(.+?)_', j).group(1))
mat = ['311','312','314','315','316','317','326','327','328','330','334','335']
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]
subj_list_table, subj_list_features, e_list = [], [], []
for file_name in files:
    paths = glob.glob(path + '/{0}/*.edf'.format(file_name))
    epochs_list = []
    j = 0
    for i in paths:
        event_id = dict(a='0')
        raw = mne.io.read_raw_edf(i)
        if 'Markers' in raw.ch_names:
           raw.drop_channels(['Markers'])
        else:
           raw.drop_channels(['Status'])
        if  len(raw.times)//250 < 10:
            continue
        new_events = mne.make_fixed_length_events(raw, id=0, start=0, duration=2, overlap=0)
        if j==1:
            epochs = mne.concatenate_epochs([mne.Epochs(raw, new_events, event_id = 0, tmin=0,
                                                        tmax=2, baseline=None, flat=dict(eeg=1e-20),
                                                        preload=True), epochs])
        else:
            epochs = mne.Epochs(raw, new_events, event_id = 0 , tmin=0, tmax=2, baseline=None,
                                flat=dict(eeg=1e-20), preload=True)
            j+=1
    epochs_list.append(epochs.copy())
    for teta in range(len(epochs_list)):
            new_names = {}
            new_names = dict(
                    (ch_name,
                     ch_name.replace('-', '').replace('Chan ', 'E').replace('CAR', '').replace('EEG ', '')
                     .replace('CA', '').replace(' ', ''))
                     for ch_name in epochs_list[teta].ch_names)
            epochs_list[teta].rename_channels(new_names)
            epochs_list[teta].set_montage(montage)
            epochs_list[teta].drop_channels(chan_drop)
    e_list.append(epochs_list)
    feat_list, tabl_list = eeg_power_band(epochs_list)
    subj_list_table.append(tabl_list)
    subj_list_features.append(feat_list)
chan1 = epochs_list[0].ch_names

In [ ]:
epochs_list

In [227]:
file_name = files[4]
paths = glob.glob(path + '/{0}/*.edf'.format(file_name))
a = 0
for i in paths:
    event_id = dict(a='0')
    raw = mne.io.read_raw_edf(i)
    a += len(raw)
print(a)

Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/315_Math_rest_open/315_Math_rest_open_n_1_m_312_t_315_eda_0_.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/315_Math_rest_open/315_Math_rest_open_n_4_m_212_t_215_eda_0_.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/315_Math_rest_open/315_Math_rest_open_n_11_m_212_t_215_eda_0_.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/ilyamikheev/Downloads/128_CHANNEL_split_REST_open_for_Ilia (314 c 319 сравнить)/315_Math_rest_open/315_Math_rest_open_n_3_m_212_t_215_eda_0_.edf...
EDF fi

In [226]:
raw.info

<Info | 7 non-empty values
 bads: []
 ch_names: EEG Chan 1-CAR, EEG Chan 2-CAR, EEG Chan 3-CAR, EEG Chan 4-CAR, ...
 chs: 129 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: 2019-03-14 15:52:34 UTC
 nchan: 129
 projs: []
 sfreq: 500.0 Hz
>

In [221]:
len(raw)

8000

In [ ]:
l_lens = np.zeros((26,1))

for i in range(len(indexes)):
      l_lens[i] = len(e_list[i][0])
e_list[0]

df_subj = pd.DataFrame(l_lens, columns=['epochs'], index=indexes)
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'subjects_epochs_128.xlsx'))
df_subj.to_excel(writer)
writer.save()



In [ ]:
len(len(e_list[i]))

In [ ]:
len(e_list[i][1])

In [ ]:
e_list

In [ ]:
subj_list_features

In [ ]:
raw.ch_names == 'Markers'


In [ ]:
raw = raw.drop_channels(['Markers'])
ch_name.replace('-Ar', '').replace('EEG ', '').replace('-A1', '').replace('-A2', ''))
                     for ch_name in epochs_list[0].ch_names)

In [ ]:
subj_list_features[1][0][0].shape

In [ ]:
e_list

In [ ]:
new_events = mne.make_fixed_length_events(raw, id=0, start=0, duration=2, overlap=0)
new_events

In [ ]:
raw.info

In [ ]:
epochs = mne.Epochs(raw, new_events, event_id = 0 , tmin=0, tmax=2, baseline=None,
                                flat=dict(eeg=1e-20), preload=True)

In [ ]:
raw = mne.io.read_raw_edf(i)
raw = raw.drop_channels(['Markers'])
raw.ch_names

In [ ]:
epochs_list[0].ch_names

In [ ]:
epochs_list[0]

In [ ]:
new_events = mne.make_fixed_length_events(raw, id=0, start=0, duration=2, overlap=0)

In [ ]:
epochs = mne.Epochs(raw, new_events, event_id = 0 , tmin=0, tmax=2, baseline=None,
                                flat=dict(eeg=1e-20), preload=True)

In [ ]:
epochs

In [ ]:
epochs.ch_names

In [ ]:
epochs_list[0].ch_names

In [ ]:
ch_names

In [ ]:
new_names

In [ ]:
epochs.ch_names

In [ ]:
raw = mne.io.read_raw_edf('/Users/ilyamikheev/Downloads/19_CHANNEL_split_REST_open_for_Ilia/001_CONTROL_male/001_CONTROL_male_n_0_m_212_t_215_eda_0_.edf')
raw.ch_names
mne.find_events(raw, stim_channel = 'Markers')

In [ ]:
raw.ch_names

In [ ]:
mne.find_events(raw, stim_channel = 'Markers',  shortest_event = 0)

In [ ]:
epochs_list[0].ch_names

In [ ]:
raw.copy().plot(start=0, duration=30)

In [ ]:
mne.events_from_annotations(raw)
montage

In [ ]:
# delete FP1, FP2
path = '/Users/ilyamikheev/Downloads/19_CHANNEL_split_REST_open_for_Ilia'
files = [f for f in sorted(os.listdir(path))]
_ = files.pop(0)
indexes = []
for i,j in enumerate(files):
    indexes.append(re.search('(.+?)_', j).group(1))
chan_drop = ['O1', 'O2']
montage = mne.channels.make_standard_montage('standard_1020')
fr_bands = {   "theta1":  [4,6],
               "theta2":  [6,8],
               "alpha1": [8,10],
               "alpha2": [10,12],
               "beta1":  [12,16],
               "beta2":  [16,20],
               "beta3":  [20,24] }
mat = ['004','008','009','011','012','013','015','016']
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]


In [ ]:
not_mat


In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
import random
list_A, list_B = [], []
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]
new_mat = random.sample(mat, 5)
new_not_mat = random.sample(not_mat, 5)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat]
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
results_group = np.zeros((5,2))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
ind = 0
for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
    A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                      for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
    B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                      for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
    C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
    C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
    x_train = np.concatenate((A,B),axis=0)
    y_train = [0]*A.shape[0] + [1]*B.shape[0]
    x_test = np.concatenate((C1, C2),axis=0)
    y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    # test size 0.2
    x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
    ac, auc, model = predict(x_train, x_test, y_train, y_test)
    for name, i in zip(['patterns_', 'filters_'],[0,1]):
        coef = get_coef(model, name, inverse_transform=True)
        coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
    results_group[count, 0] = ac
    results_group[count, 1] = auc


In [ ]:
mat

In [ ]:
mat = ['001','004','008','009','011','012','013','015','016']
not_mat = [x for x in indexes if x not in mat]

In [ ]:
mat

In [ ]:
not_mat

In [239]:
import random
list_A, list_B = [], []
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]
new_mat = ['314', '326', '327', '328', '335', '315']
new_not_mat = ['318','319','320','322','333','329']
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat]
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
results_group = np.zeros((6,2))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
ind = 0
for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
    A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                      for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
    B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                      for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
    C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
    C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
    x_train = np.concatenate((A,B),axis=0)
    y_train = [0]*A.shape[0] + [1]*B.shape[0]
    x_test = np.concatenate((C1, C2),axis=0)
    y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    # test size 0.2
    x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
    sc = StandardScaler()
    sc.fit(x_train)
    x_train = sc.transform(x_train)
    x_test = sc.transform(x_test)
    lgb = LGBMClassifier(objective='binary')
    lgb.fit(x_train, y_train)
    predictions = lgb.predict_proba(x_test)
    auc = roc_auc_score(y_test, predictions[:,1])
    ac = accuracy_score(y_test, lgb.predict(x_test))
    results_group[count, 0] = ac
    results_group[count, 1] = auc


In [241]:
import random
list_A, list_B = [], []
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]
new_mat = ['314', '326', '327', '328', '335', '315']
new_not_mat = ['318','319','320','322','333','329']
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat]
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
results_group = np.zeros((6,2))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
ind = 0
for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
    A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                      for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
    B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                      for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
    C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
    C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
    x_train = np.concatenate((A,B),axis=0)
    y_train = [0]*A.shape[0] + [1]*B.shape[0]
    x_test = np.concatenate((C1, C2),axis=0)
    y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    # test size 0.2
    x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
    ac, auc = predict(x_train, x_test, y_train, y_test)
    results_group[count, 0] = ac
    results_group[count, 1] = auc

NameError: name 'presict' is not defined

In [236]:
results_group.shape

index=[[i,j] for i, j in zip(new_mat, new_not_mat)]

index

index=[f'{i}_{j}' for i, j in zip(new_mat, new_not_mat)]
index

['311_313', '312_324', '316_331', '317_332', '330_337', '334_336']

In [240]:
datafr_groups = pd.DataFrame(results_group, columns=['ac', 'auc'], index=[f'{i}_{j}' for i, j in zip(new_mat, new_not_mat)] )
datafr_groups.to_excel(os.path.join('/Users/ilyamikheev/Downloads/', 'classification_catboost_128_not_suc.xlsx'), index = True)

In [ ]:
e_list

In [ ]:
new_mat

In [ ]:
new_not_mat

In [ ]:
e_list

In [ ]:
indexes

In [ ]:
# classification mathematicians/non mathematicians
results_group,  = np.zeros((4,1)), np.zeros((4,2,len(fr_bands),len(chan1)))
list_A, list_B, list_C_1, list_C_2 = [], [], [], []
for ind in range(3):
    if ind < 3:
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[:-1]][i][ind],axis = 1)
                              for i in range(len(mat[:-1]))]),axis=0)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[:-1]][i][ind],axis = 1)
                              for i in range(len(not_mat[:-1]))]),axis=0)
        #C1 = np.stack(subj_list_features[index_mat[-1]][ind], axis = 1)
        C2 = np.stack(subj_list_features[index_not_mat[-1]][ind], axis = 1)
    y_train = ['0']*A.shape[0] + ['1']*B.shape[0]
    #y_test = ['0']*C1.shape[0] + ['1']*C2.shape[0]
    y_test = ['0']*C2.shape[0]
    x_train = np.concatenate((A,B),axis=0)
    #x_test = np.concatenate((C1,C2),axis=0)
    x_test = C2
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    x_train, x_test_1, y_train, y_test_1 = train_test_split(x_train, y_train, test_size=0.1)
    results_group[ind,...], model = predict(x_train, x_test, y_train, y_test)
    for name, i in zip(['patterns_', 'filters_'],[0,1]):
        coef = get_coef(model, name, inverse_transform=True)
        coefs_group[ind,i,...] = coef.reshape(len(fr_bands),-1)

In [ ]:
mat


In [ ]:
e_list

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

In [ ]:
indexes

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def predict_c(x_train, x_test, y_train, y_test):
    csp = CSP(n_components=25, reg=None, log=True, norm_trace=False)
    results = np.zeros((1,4))
    model = make_pipeline(csp,                  
                               LinearModel(LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='roc_auc',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10 
                                                                       ))) 
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    ac = accuracy_score(y_test,y_predict)
    score = model.score(x_test,y_test)
    #cm=confusion_matrix(y_test,y_predict)
    #TN,TP,FN,FP = cm[1,1],cm[0,0],cm[1,0],cm[0,1]
    #results[0,0], results[0,1] = accuracy_score(y_test,y_predict), 0
    #results[0,2], results[0,3] = TP/float(TP+FN), TN/float(TN+FP)
    return ac, score

In [ ]:
array([[21, 2],
       [4, 25]])


In [ ]:
results = np.zeros((len(indexes),9))
for subj in tqdm(range(len(indexes))):  
    k = 3
    for i, key in enumerate(list(dict_cls.keys())):
        ind = dict_cls[key]
        A = e_list[subj][ind[0]].get_data()
        B = e_list[subj][ind[1]].get_data()
        epochs_data_train = np.concatenate([A, B])
        labels =  [0]*A.shape[0] + [1]*B.shape[0]
        cv_split = cv.split(epochs_data_train)
        labels = np.array(labels)
        x_train, x_test, y_train, y_test = train_test_split(epochs_data_train, labels, test_size=0.1)
        # Assemble a classifier

        # Use scikit-learn Pipeline with cross_val_score function
        #clf = Pipeline([('CSP', csp), ('LDA', lda)])
        #accuracy = np.mean(cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring = 'accuracy'))
        #score = np.mean(cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1))
        accuracy, score = predict(x_train, x_test, y_train, y_test)
        # Printing the results
        class_balance = np.mean(labels == labels[0])
        class_balance = max(class_balance, 1. - class_balance)
        results[subj,k-3:k] = accuracy, score, class_balance
        k+=3


In [ ]:
len(e_list)


In [ ]:
A = e_list[11][0].get_data()
B = e_list[11][1].get_data()
epochs_data_train = np.concatenate([A, B])
labels =  [0]*A.shape[0] + [1]*B.shape[0]
cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)
labels = np.array(labels)
x_train, x_test, y_train, y_test = train_test_split(epochs_data_train, labels, test_size=0.1)
# Assemble a classifier
csp = CSP(n_components=10, reg=None, log=True, norm_trace=False)
lda = LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='f1_score',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10)
# Use scikit-learn Pipeline with cross_val_score function
#clf = Pipeline([('CSP', csp), ('LDA', lda)])
#accuracy = np.mean(cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1, scoring = 'accuracy'))
#score = np.mean(cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1))
accuracy, score = predict_c(x_train, x_test, y_train, y_test)
# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)

In [ ]:
y_test

In [ ]:
results

In [ ]:
labels

In [ ]:
score

In [ ]:
y_test

In [ ]:
y_train

In [ ]:
accuracy

In [ ]:
class_balance

In [ ]:
results.mean(axis=0)

In [ ]:
# save results to table
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin, 
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'subjects_classification_csp_final.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()

In [ ]:
list_gr, ar_mean_var = [results,results[index_mat,...],results[index_not_mat,...]], np.zeros((3,18))
# roc auc
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC','CB']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name 
                                                                 for j in ['mean','var'] ]])

In [ ]:
accuracy

In [ ]:
list_gr, ar_mean_var = [results,results[index_mat,...],results[index_not_mat,...]], np.zeros((3,18))
# roc auc
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC','CB']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name 
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin, 
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'subjects_classification_csp_final.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()                                                               for j in ['mean','var'] ]])

In [ ]:
score

In [ ]:
np.mean(labels == labels[0])

In [ ]:
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
labels == 0

In [ ]:
LinearModel(LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='roc_auc',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10 

In [ ]:
labels[0]

In [ ]:
labels

In [ ]:
np.mean(labels == labels[0])

In [ ]:
class_balance

In [ ]:
labels

In [ ]:
epochs_data_train.shape

In [ ]:
labels

In [ ]:
len(epochs_list)

In [ ]:
# info about chan positions
info = epochs_list[0].info

In [ ]:
type(subj_list_table)

In [ ]:
type(info)

In [ ]:
info

In [ ]:
bb = subj_list_features.copy()

In [ ]:
bb = np.array(bb)

In [ ]:
bb[0].shape

In [ ]:
type(subj_list_features)

In [ ]:
info1 = mne.create_info(info.ch_names, ch_types= 'eeg',
                        sfreq = 250)

In [ ]:
info1.set_montage(montage)

In [ ]:
a1 = subj_list_table.copy()

In [ ]:
a5 = subj_list_features.copy()

In [ ]:
a1=np.array(a1)

In [ ]:
a1.shape

In [ ]:
a2 = a1[:,1,...]

In [ ]:
a5.shape

In [ ]:
len(a5[0][1][0])

In [ ]:
C1 = np.stack(a5[0][0], axis = 1)

In [ ]:
a3

In [ ]:
C1.shape

In [ ]:
type(a1)

In [ ]:
a5 = np.array(a5[0])

In [ ]:
a2 = a1[:,0,...]

In [ ]:
a2.shape

In [ ]:
a3 = a2[:,6,:]

In [ ]:
a2.shape

In [ ]:
mat = ['311','312','314','315','316','317','326','327','328','330','334','335']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat]

In [ ]:
mat = ['311','312','314','315','316','317','326','327','328','330','334','335']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)

In [ ]:
f3.shape

In [ ]:
b1 = a3[index_mat]
b2 = a3[index_not_mat]

In [ ]:
b1 = a1[:,0,...]
b2 = a1[:,1,...]

In [ ]:
a1.shape

In [ ]:
b1 = b1[index_mat]
b2 = b2[index_mat]

In [ ]:
b1.shape

In [ ]:
b2.shape

In [ ]:
b1.shape

In [ ]:
b2.shape

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne.time_frequency import tfr_morlet
from mne.stats import permutation_cluster_test
from mne.datasets import sample

In [ ]:
T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([b1, b2], threshold = 1, out_type='mask',
                             n_permutations=1000)

In [ ]:
cluster_p_values

In [ ]:
C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)

In [ ]:
cluster_p_values

In [ ]:
a1.shape

In [ ]:
b1.shape

In [ ]:
a2.shape

In [ ]:
index_mat

In [ ]:
res_ar = np.zeros_like(a1[0,0,...])
a2 = a1[:,0,...]
#mr_ar 
ll = []
for i in range(7):
    a3 = a2[:,i,:]
    b1 = a3[index_mat]
    b2 = a3[index_not_mat]
    #b1 = a1[:,0,i,...]
    #b2 = a1[:,2,i,...]
    #c1 = np.stack(a5[0][0], axis = 1)
    #c2 = np.stack(a5[0][1], axis = 1)
    #b1 = c1[:,i,:]
    #b2 = c2[:,i,:]
    #b1 = b1[index_mat]
    #b2 = b2[index_mat]
    #b1 = a3[index_mat]
    #b2 = a3[index_not_mat]
    T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_test([b1, b2], threshold = 1, out_type='mask',
                             n_permutations=1000)
    T_obs_plot = np.nan * np.ones_like(T_obs)
    for c, p_val in zip(clusters, cluster_p_values):
        if p_val <= 0.05:
            T_obs_plot[c] = T_obs[c]
    np.nan_to_num(T_obs_plot, copy=False)
    if np.count_nonzero(T_obs_plot) > 0:
        ll.append(i)
        res_ar[i,:] = T_obs_plot
mr_ar = res_ar>0
    

In [ ]:
res_ar

In [ ]:
#without outl
mat = ['311','312','315','316','317','327','328','330','334']  
#not_mat = ['318','320','322','323','325','329','331','333']
not_mat = ['320','322','323','325','329','331','333']
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:
from matplotlib import cm

In [ ]:
import matplotlib.colors as mcolors
norm = mcolors.DivergingNorm(vmin = -np.amax(res_ar), vmax = np.amax(res_ar), vcenter=0)

In [ ]:
# с 6 до 13 HZ
# DFA

def plot_topo(ar, info):
    #vmin = np.amin(ar)
    vmin = 0
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=1, ncols=len(ll), figsize=(30, 20))
    ss = 0
    for i, key in enumerate(list(fr_bands.keys())):
        if i in ll:
            evoked = mne.EvokedArray(ar[i,:].reshape(-1,1),
                             info1, tmin=0.)
            a = evoked.plot_topomap(axes=axes[ss],time_format=None, colorbar = False,  times=[0], mask = mr_ar[i,:].reshape(-1,1), size = 5, show_names = True, 
                        show = False, vmin = 0, cmap='Reds')
            axes[ss].set_title(label = '{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout()
            ss+=1
    m = cm.ScalarMappable(cmap='Reds')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    #plt.figtext(0.5, 1.0, '{}'.format('Sagnificant clusters'), va="center", ha="center", size=60, fontweight = 'semibold')
    return fig

In [ ]:
fig = plot_topo(res_ar, info)

In [ ]:
%matplotlib inline
#fig = plot_topo(T_obs_plot, info)
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads', 'cluster_bands_244.png'), format='png', bbox_inches='tight', dpi=600)
#plt.save(os.path.join(path_subj_topo, 'cluster_test_244_fin_3.png'), format='png', dpi=600)
plt.close(fig)

In [ ]:
def plot_topo(ar):
    vmin = np.amin(ar)
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(30, 20)) 
    for name, pos, plot_name, ind in zip(('patterns_', 'filters_'),(0.8,0.5),
                                         ('Patterns','Filters'),(0,1)):
        for i,key in enumerate(list(fr_bands.keys())):
            a = mne.viz.plot_topomap(ar[ind,i,:],info,vmin=vmin,vmax=vmax, axes=axes[ind,i], 
                                 show = False)
            axes[ind,i].set(title='{}-{} Hz'.format(*fr_bands[key]))
            mne.viz.tight_layout() 
        plt.figtext(0.5,pos,'{}'.format(plot_name), va="center", ha="center", size=24, fontweight = 'semibold',)
    return fig

In [ ]:
from matplotlib import cm

In [ ]:
b2.shape

In [ ]:
cluster_p_values

In [ ]:
mat = ['311','312','315','316','317','327','328','330','334']  
not_mat = ['320','322','323','325','329','331','333']
new_mat = mat
new_not_mat = not_mat
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 

In [ ]:
imp = np.concatenate([f1,f2],axis=0)  

In [ ]:
# write relative powers to table
df1,df2 = {}, {}
k, l = 0, 0
for i in range(len(subj_list_table[0])):
    for j in range(len(subj_list_table[0][0])):
        df1[k] = np.array(subj_list_table)[:,i,j,:]
        k += 1
for s in list(df1.keys()):      
    df2[s] = pd.DataFrame(columns=chan1, index=indexes)
    for ind_num,ind_name in enumerate(indexes):
        df2[s].loc['{}'.format(ind_name)] = pd.Series(df1[s][ind_num,:], chan1)
writer = pd.ExcelWriter(os.path.join(path_res, 'subjects_relative_power.xlsx'), engine='xlsxwriter')
for ind in events_list:
    for band_name in list(fr_bands.keys()):
        df2[l].to_excel(writer, sheet_name='{},({},{})'.format(ind, *fr_bands[band_name]))
        l += 1
writer.save()

In [ ]:
# classification for each subject in both groups
results, coefs = np.zeros((len(indexes),12)), np.zeros((len(indexes),len(dict_cls),2,len(fr_bands),len(chan1))) 
for subj in range(len(indexes)):
# change to zip
    k = 4             
    for i, key in enumerate(list(dict_cls.keys())):
        ind = dict_cls[key]
        A, B = np.stack(subj_list_features[subj][ind[0]],axis=1), np.stack(subj_list_features[subj][ind[1]],axis=1)
        y = np.array(['0']*A.shape[0] + ['1']*B.shape[0]) 
        x = np.concatenate((A,B),axis=0)
        x_train, x_test, y_train, y_test = train_test_split(x.reshape(x.shape[0],-1),y,test_size=0.3)
        results[subj,k-4:k], model = predict(x_train, x_test, y_train, y_test)
        for name, j in zip(['patterns_', 'filters_'],[0,1]):
            coef = get_coef(model, name, inverse_transform=True)
            coefs[subj,i,j,...] = coef.reshape(len(fr_bands),-1)
        k+=4

In [ ]:
dict_cls

In [ ]:
# evaluate mean and var for all subjects/and for subjects in groups
list_gr, ar_mean_var = [results,results[index_mat,...],results[index_not_mat,...]], np.zeros((3,24))
# roc auc
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC','TPR','TNR']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name 
                                                                 for j in ['mean','var'] ]])

In [ ]:
# save results to table
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin, 
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join(path_subj, 'subjects_classification_l1_final.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()

In [ ]:
# plot topomaps for each subject
for i, subj_ind in enumerate(indexes):
    for j, key in enumerate(list(dict_cls.keys())):
        fig = plot_topo(coefs[i,j])
        fig.savefig(os.path.join(path_subj_topo, 'filters_patterns_{}_{}.png'
                                 .format(subj_ind,['241_244','242_244','241_242'][j])), format='png', dpi=600)
        plt.close(fig)

In [ ]:
# classification mathematicians/non mathematicians
results_group,  = np.zeros((4,1)), np.zeros((4,2,len(fr_bands),len(chan1)))
list_A, list_B, list_C_1, list_C_2 = [], [], [], []
for ind in range(3):
    if ind < 3:
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[:-1]][i][ind],axis = 1)
                              for i in range(len(mat[:-1]))]),axis=0)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[:-1]][i][ind],axis = 1)
                              for i in range(len(not_mat[:-1]))]),axis=0)
        #C1 = np.stack(subj_list_features[index_mat[-1]][ind], axis = 1)
        C2 = np.stack(subj_list_features[index_not_mat[-1]][ind], axis = 1)
    y_train = ['0']*A.shape[0] + ['1']*B.shape[0] 
    #y_test = ['0']*C1.shape[0] + ['1']*C2.shape[0] 
    y_test = ['0']*C2.shape[0]
    x_train = np.concatenate((A,B),axis=0)
    #x_test = np.concatenate((C1,C2),axis=0)
    x_test = C2
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = x_test.reshape(x_test.shape[0],-1)
    x_train, x_test_1, y_train, y_test_1 = train_test_split(x_train, y_train, test_size=0.1)
    results_group[ind,...], model = predict(x_train, x_test, y_train, y_test)
    for name, i in zip(['patterns_', 'filters_'],[0,1]):
        coef = get_coef(model, name, inverse_transform=True)
        coefs_group[ind,i,...] = coef.reshape(len(fr_bands),-1)

In [ ]:
#without outl
mat = ['311','312','315','316','317','327','328','330','334']  
#not_mat = ['318','320','322','323','325','329','331','333']
not_mat = ['320','322','323','325','329','331','333']
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:

mat = ['311','312','314','315','316','317','326','327','328','330','334','336']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:
mat = ['311','312','313','315','317','319','324','327','328','330','332','334','336','337']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 

In [ ]:
results_group, coefs_group = np.zeros((4,4)), np.zeros((4,2,len(fr_bands),len(chan1)))
list_A, list_B, list_C_1, list_C_2 = [], [], [], []
A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[3:]][i][ind],axis = 1)
                      for i in range(len(mat[3:]))]),axis=0)
B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[3:]][i][ind],axis = 1)
                      for i in range(len(not_mat[3:]))]),axis=0)
C1 = np.stack(subj_list_features[index_mat[2]][ind], axis = 1)
C2 = np.stack(subj_list_features[index_not_mat[2]][ind], axis = 1)
y_train = [0]*A.shape[0] + [1]*B.shape[0] 
#y_test = [0]*C1.shape[0] + [1]*C2.shape[0] 
y_test = [0]*C1.shape[0]
x_train = np.concatenate((A,B),axis=0)
#x_test = np.concatenate((C1,C2),axis=0)
x_test = C1
x_train = x_train.reshape(x_train.shape[0],-1)
x_test = x_test.reshape(x_test.shape[0],-1)
x_train, x_test_1, y_train, y_test_1 = train_test_split(x_train, y_train, test_size=0.1)
#results_group[ind,...], model = predict(x_train, x_test, y_train, y_test)
#coef = get_coef(model, 'patterns_', inverse_transform=True)

In [ ]:
def predict_subject(X, y, Pattern, inter):
    #X = X - X.mean(0, keepdims=True)
    #scaler = StandardScaler()
    #scaler.fit(X)
    #scaler.transform(X)
    #scaler.transform(Pattern)
    #scaler.transfrom(inter)
    prec_x = np.linalg.pinv(np.cov(X.T))
    #Pattern = Pattern.ravel()
    new_filter = prec_x.dot(Pattern.T).T 
    scores = np.dot(X, new_filter.T) + inter
    scores.ravel()
    #scores_i = 1/(1+np.exp(scores))
    #y_predict = [1 if i > 0.2 else 0 for i in scores_i]
    indices = (scores > 0).astype(int) 
    cl = np.array([0,1])
    y_predict = cl[indices]
    a_s = accuracy_score(y, y_predict)
    return a_s

In [ ]:
# succes in solving tasks
#results_group = np.zeros((30,4))
list_A, list_B = [], []
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
#mat = ['311','312','314','315','316','317','326','327','328','330','334','335']  
#not_mat = ['313','319','320','322','323','324','325','329','331','332','333', '336', '337']
mat = ['311','312','313','315','317','319','324','327','328','330','332','334','336','337']  
not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 
#not_mat = ['320','322','323','325','329','331','333']
new_mat = random.sample(mat, 10)
new_not_mat = random.sample(not_mat, 10)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 
all_indexes = index_mat + index_not_mat
results_group = np.zeros((10,6))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for ind in range(4):
    if ind < 3:
        for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
            A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                                  for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
            B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                                  for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
            C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
            C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2),axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
            results_group[count, ind] = ac
            results_group[count, ind+3] = auc
    else:
         pass

In [ ]:
import random

In [ ]:
#mathematicians/not mathematicians
#list_A, list_B = [], []
#all_indexes = index_mat + index_not_mat
#results_group = np.zeros((len(all_indexes),4))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
#index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
list_A, list_B = [], []
#mat = ['311','312','314','315','316','317','326','327','328','330','334','335']
#not_mat = [x for x in indexes if x not in mat]
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 
new_mat = random.sample(mat, 8)
new_not_mat = random.sample(not_mat, 8)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
results_group = np.zeros((10,6))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for ind in range(4):
    if ind < 3:
        for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
            A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                                  for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
            B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                                  for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
            C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
            C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2),axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            # test size 0.2
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
            results_group[count, ind] = ac
            results_group[count, ind+3] = auc
    else: 
         pass

In [ ]:
results_group[:8]

In [ ]:
#suc/not suc
#list_A, list_B = [], []
#all_indexes = index_mat + index_not_mat
#results_group = np.zeros((len(all_indexes),4))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
#index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
list_A, list_B = [], []
mat = ['311','312','317','324','327','330','332','334','336', '337']  
not_mat = ['314','316','318','320','322','326','329','333','335', '320']
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 
new_mat = random.sample(mat, 10)
new_not_mat = random.sample(not_mat, 10)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,10,2,len(fr_bands),len(chan1)))
results_group = np.zeros((10,6))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for ind in range(4):
    if ind < 3:
        for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
            A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                                  for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
            B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                                  for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
            C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
            C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2),axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            # test size 0.2
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
            results_group[count, ind] = ac
            results_group[count, ind+3] = auc
    else: 
         pass

In [ ]:
all_indexes

In [ ]:
new_mat

In [ ]:
new_not_mat

In [ ]:
index_mat

In [ ]:
results_group

In [ ]:
results_group.shape

In [ ]:
datafr_groups = pd.DataFrame(tt.reshape((1,6)), columns=['241_ac', '242_ac', '244_ac', '241_auc', '242_auc', '244_auc'])
datafr_groups.to_excel(os.path.join('/Users/ilyamikheev/Downloads/', 'suc_classification_quality_two_subjects_2_seconds_std.xlsx'), index = False)

In [ ]:
tt = np.average(results_group, axis=0)

In [ ]:
tt

In [ ]:
results_group

In [ ]:
new_mat

In [ ]:
tt

In [ ]:
pp = coefs_group[0,:].mean(axis = (2))

In [ ]:
coefs_group[0,...].shape

In [ ]:
pp.shape

In [ ]:
from matplotlib import cm
fig = plot_patterns(coefs_group[2,...].mean(axis = 0))

In [ ]:
fig.savefig(os.path.join('/Users/ilyamikheev/Downloads/', 'filters_patterns_mat_not_mat.png'), format='png', dpi=600, bbox_inches='tight')

In [ ]:
from matplotlib import cm
fig = plot_patterns(coefs_group[2,0])

In [ ]:
def plot_patterns(ar):
    vmin = - np.amax(ar)
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(30, 20)) 
    for name, pos, plot_name, ind in zip(('patterns_', 'filters_'),(0.82,0.5),
                                         ('Patterns','Filters'),(0,1)):
        for i,key in enumerate(list(fr_bands.keys())):
            a = mne.viz.plot_topomap(ar[ind,i,:],info,vmin=vmin,vmax=vmax, axes=axes[ind,i], 
                                 show = False)
            axes[ind,i].set_title(label='{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout() 
        plt.figtext(0.5,pos,'{}'.format(plot_name), va="center", ha="center", size=44, fontweight = 'semibold')
    m = cm.ScalarMappable(cmap='RdBu_r')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    return fig

In [ ]:
cax = fig.add_axes([1, 0.3, 0.03, 0.38])

In [ ]:
pos = fig.add_axes([0.93,0.1,0.02,0.35])

In [ ]:
coefs_group.shape

In [ ]:
index_mat

In [ ]:
results_group

In [ ]:
list_gr, ar_mean_var = [results,results[index_mat,...],results[index_not_mat,...]], np.zeros((3,18))
# roc auc
list_names_subj, list_names_fin = [], []
metr_name = ['accuracy','ROC/AUC','CB']
for i, key in enumerate(list(dict_cls.keys())):
    ar_mean_var[i,...] = np.array(list(chain.from_iterable((a, b) for a,b in zip(list_gr[i].mean(axis=0),
                                                                                 list_gr[i].var(axis=0)))))
    list_names_subj.extend(['{} {}'.format(key,name) for name in metr_name])
    list_names_fin.extend(['{} {}'.format(key,name) for name in ['{} {}'.format(i,j) for i in metr_name 
df_subj = pd.DataFrame(results, columns=list_names_subj, index=indexes)
df_fin = pd.DataFrame(ar_mean_var, columns=list_names_fin, 
                       index=['all_subjects','mathematicians','not_mathematicians'])
writer = pd.ExcelWriter(os.path.join('/Users/ilyamikheev/Downloads', 'subjects_classification_csp_final.xlsx'), engine='xlsxwriter')
df_subj.to_excel(writer, sheet_name='all_subj')
df_fin.to_excel(writer, sheet_name='mean_var')
writer.save()     

In [ ]:
l_names = ['{}_{}'.format(a,b) for a,b in zip(new_mat,new_not_mat)]

In [ ]:
l_names

In [ ]:
datafr_groups = pd.DataFrame(results_group, index = l_names, columns=['241_ac', '242_ac', '244_ac', '241_auc', '242_auc', '244_auc'])
datafr_groups.to_excel(os.path.join('/Users/ilyamikheev/Downloads/', 'group_2_subjects_2_seconds_suc_re.xlsx'))

In [ ]:
results_group.shape

In [ ]:
for i in range(4):
    fig = plot_topo(coefs_group[i,...])
    fig.savefig(os.path.join(path_group_topo, 'filters_patterns_mat_not_mat_{}.png'
                             .format(['241','242','244','all'][i])), format='png', dpi=600)
    plt.close(fig)

In [ ]:
tt = np.average(results_group, axis=0)

In [ ]:
tt.shape

In [ ]:
a = np.array([0,1])
b = np.array([2,3])
np.concatenate([a,b])


In [ ]:
tt1 = results_group.std(axis=0)

In [ ]:
tt1

In [ ]:
tt = tt.reshape(6,1)

In [ ]:
datafr_groups = pd.DataFrame([tt1], columns=['241_ac', '242_ac', '244_ac', '241_auc', '242_auc', '244_auc'])
datafr_groups.to_excel(os.path.join('/Users/ilyamikheev/Downloads/', 'subj_in_groups_classification_quality_two_subjects_2_seconds_std.xlsx'), index = False)

In [ ]:
index_mat, index_not_mat = [indexes.index(i) for i in mat], [indexes.index(i) for i in not_mat] 
new_mat = random.sample(mat, 5)
new_not_mat = random.sample(not_mat, 5)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 
all_indexes = index_mat + index_not_mat
coefs_group = np.zeros((4,5,2,len(fr_bands),len(chan1)))
results_group = np.zeros((5,6))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for ind in range(4):
    if ind < 3:
        for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
            A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                                  for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
            B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                                  for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
            C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
            C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2),axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            # test size 0.2
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
            results_group[count, ind] = ac
            results_group[count, ind+3] = auc
    else: 
         pass

In [ ]:
dict_cls

In [ ]:
import random

In [ ]:
# classification using all subjects in both groups and all subjects
index_mat_n, index_not_mat_n = index_mat, index_not_mat
results_all, coefs_all = np.zeros((3,30,6)), np.zeros((3,len(dict_cls),2,len(fr_bands),len(chan1)))
list_of_len = [len(indexes), len(index_mat), len(index_not_mat), len(index_not_mat)]
for group_ind, subj_type in enumerate([[*range(len(indexes))],index_mat,index_not_mat]):
    subj_type = np.array(subj_type)
    results_sgroup = np.zeros((list_of_len[group_ind], 6))
    coefs_group_s = np.zeros((len(dict_cls),list_of_len[group_ind],2,len(fr_bands),len(chan1)))
    for i, key in enumerate(list(dict_cls.keys())):
        ind = dict_cls[key]
        for count, p in enumerate(subj_type): 
            A = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[0]],axis = 1) 
                                for it in range((subj_type.shape[0])-1)]),axis=0)
            B = np.concatenate(([np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][it][ind[1]],axis = 1) 
                                for it in range(subj_type.shape[0]-1)]),axis=0)
            C1 = np.stack(subj_list_features[p][ind[0]],axis = 1) 
            C2 = np.stack(subj_list_features[p][ind[1]],axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2), axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]  
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i1 in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group_s[i,count,i1,...] = coef.reshape(len(fr_bands),-1)
            results_sgroup[count, i] = ac
            results_sgroup[count, i+3] = auc
    #m_m = np.average(results_sgroup, axis=0)
    #m_v = results_sgroup.std(axis=0)
    coefs_m = coefs_group_s.mean(axis = 1)
    coefs_all[group_ind,...] = coefs_m
    results_all[group_ind,...] = results_sgroup

In [ ]:
datafr_groups = pd.DataFrame(results_sgroup, index = indexes , columns=['241/244_ac', '242/244_ac', '241/242_ac', '241/244_auc', '242/244_auc', '241/242_auc'])
datafr_groups.to_excel(os.path.join('/Users/ilyamikheev/Downloads/', 'group_wise_2_seconds_powers.xlsx'))

In [ ]:
results_sgroup

In [ ]:
indexes

In [ ]:
m_m.shape

In [ ]:
len(subj_list_features[0])

In [ ]:
len(subj_list_features[0][0])

In [ ]:
i

In [ ]:
auc

In [ ]:
results_sgroup[count, ind]

In [ ]:
np.stack([subj_list_features][p][i][ind[0]],axis = 1).shape

In [ ]:
[np.stack([subj_list_features[j] for j in subj_type[subj_type!=p]][i][ind[0]],axis = 1) 
                                for i in range(len(subj_type)-1)]

In [ ]:
subj_type[subj_type!=p]

In [ ]:
subj_type[subj_type!=p]

In [ ]:
subj_type

In [ ]:
p

In [ ]:
ind

In [ ]:
len(subj_list_features[0][1][0])

In [ ]:
len(subj_list_features[0][0])

In [ ]:
np.concatenate([np.stack([subj_list_features][0][i][ind[1]],axis = 1) 
                                for i in range(1)]).shape

In [ ]:
results_all

In [ ]:
a, b, c = predict(x_train, x_test, y_train, y_test)

In [ ]:
results_all

In [ ]:
def predict(x_train, x_test, y_train, y_test):
    results = np.zeros((1,4))
    model = make_pipeline( StandardScaler(),                  
                               LinearModel(LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='roc_auc',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10 
                                                                       ))) 
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    ac = accuracy_score(y_test,y_predict)
    score = model.score(x_test,y_test)
    #cm=confusion_matrix(y_test,y_predict)
    #TN,TP,FN,FP = cm[1,1],cm[0,0],cm[1,0],cm[0,1]
    #results[0,0], results[0,1] = accuracy_score(y_test,y_predict), 0
    #results[0,2], results[0,3] = TP/float(TP+FN), TN/float(TN+FP)
    return ac, score, model

In [ ]:
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for ind in range(4):
    if ind < 3:
        for count, (s_ind_1, s_ind_2) in enumerate(zip(index_mat, index_not_mat)):
            A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind_1]][i][ind],axis = 1)
                                  for i in range(index_mat[index_mat!=s_ind_1].shape[0])]),axis=0)
            B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind_2]][i][ind],axis = 1)
                                  for i in range(index_not_mat[index_not_mat!=s_ind_2].shape[0])]),axis=0)
            C1 = np.stack(subj_list_features[s_ind_1][ind], axis = 1)
            C2 = np.stack(subj_list_features[s_ind_2][ind], axis = 1)
            x_train = np.concatenate((A,B),axis=0)
            y_train = [0]*A.shape[0] + [1]*B.shape[0] 
            x_test = np.concatenate((C1, C2),axis=0)
            y_test = [0]*C1.shape[0] + [1]*C2.shape[0]
            x_train = x_train.reshape(x_train.shape[0],-1)
            x_test = x_test.reshape(x_test.shape[0],-1)
            x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
            ac, auc, model = predict(x_train, x_test, y_train, y_test)
            for name, i in zip(['patterns_', 'filters_'],[0,1]):
                coef = get_coef(model, name, inverse_transform=True)
                coefs_group[ind,count,i,...] = coef.reshape(len(fr_bands),-1)
            results_group[count, ind] = ac
            results_group[count, ind+3] = auc
    else:
         pass

In [ ]:
results_all.shape

In [ ]:
results_group

In [ ]:
def predict(x_train, x_test, y_train, y_test):
    results = np.zeros((1,4))
    model = make_pipeline( StandardScaler(),                  
                               LinearModel(LogisticRegressionCV(
                               Cs=list(np.power(10.0, np.arange(-10, 10))),
                               penalty='l2',
                               scoring='roc_auc',
                               random_state=0,
                               max_iter=10000,
                               fit_intercept=True,
                               solver='newton-cg',
                               class_weight='balanced',
                               tol=10 
                                                                       ))) 
    # create method fr sign weights extraction  
    model.fit(x_train, y_train)
    y_predict = model.predict(x_test)
    ac = accuracy_score(y_test,y_predict)
    score = model.score(x_test,y_test)
    #cm=confusion_matrix(y_test,y_predict)
    #TN,TP,FN,FP = cm[1,1],cm[0,0],cm[1,0],cm[0,1]
    #results[0,0], results[0,1] = accuracy_score(y_test,y_predict), 0
    #results[0,2], results[0,3] = TP/float(TP+FN), TN/float(TN+FP)
    
    return ac, score, model 
# plot topomap for each freq band
def plot_patterns(ar):
    vmin = - np.amax(ar)
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=2, ncols=7, figsize=(30, 20)) 
    for name, pos, plot_name, ind in zip(('patterns_', 'filters_'),(0.82,0.5),
                                         ('Patterns','Filters'),(0,1)):
        for i,key in enumerate(list(fr_bands.keys())):
            a = mne.viz.plot_topomap(ar[ind,i,:],info,vmin=vmin,vmax=vmax, axes=axes[ind,i], 
                                 show = False)
            axes[ind,i].set_title(label='{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout() 
        plt.figtext(0.5,pos,'{}'.format(plot_name), va="center", ha="center", size=44, fontweight = 'semibold')
    m = cm.ScalarMappable(cmap='RdBu_r')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    return fig

In [ ]:
np.amin(coefs_group[0,8])

In [ ]:
fig = plot_patterns(coefs_group[2,4])

In [ ]:
%matplotlib inline
#fig = plot_topo(T_obs_plot, info)
fig.savefig(os.path.join(path_subj_topo, 'group_classif_244.png'), format='png', bbox_inches='tight', dpi=600)
#plt.save(os.path.join(path_subj_topo, 'cluster_test_244_fin_3.png'), format='png', dpi=600)
plt.close(fig)

In [ ]:
def plot_topo(ar, info):
    #vmin = np.amin(ar)
    vmin = 0
    vmax = np.amax(ar)
    fig, axes = plt.subplots(nrows=1, ncols=len(ll), figsize=(30, 20))
    ss = 0
    for i, key in enumerate(list(fr_bands.keys())):
        if i in ll:
            evoked = mne.EvokedArray(ar[i,:].reshape(-1,1),
                             info, tmin=0.)
            a = evoked.plot_topomap(axes=axes[ss],time_format=None, colorbar = False,  times=[0], mask = mr_ar[i,:].reshape(-1,1), size = 5, show_names = True, 
                        show = False, vmin = 0, cmap='Reds')
            axes[ss].set_title(label = '{}-{} Hz'.format(*fr_bands[key]), fontdict = {'fontsize': 40, 'fontweight' : 'semibold'})
            mne.viz.tight_layout()
            ss+=1
    m = cm.ScalarMappable(cmap='Reds')
    m.set_array([vmin, vmax])
    cax = fig.add_axes([1, 0.3, 0.03, 0.38])
    cb = fig.colorbar(m, cax)
    cb.ax.tick_params(labelsize=40)
    #plt.figtext(0.5, 1.0, '{}'.format('Sagnificant clusters'), va="center", ha="center", size=60, fontweight = 'semibold')
    return fig

In [ ]:
from matplotlib import cm

In [ ]:
%matplotlib inline
#fig = plot_topo(T_obs_plot, info)
fig.savefig(os.path.join(path_subj_topo, 'cluster_bands_244.png'), format='png', bbox_inches='tight', dpi=600)
#plt.save(os.path.join(path_subj_topo, 'cluster_test_244_fin_3.png'), format='png', dpi=600)
plt.close(fig)

In [ ]:
datafr_groups = pd.DataFrame(results_group[:,:], columns=['241_ac', '242_ac', '244_ac', '241_auc', '242_auc', '244_auc'])
datafr_groups.to_excel(os.path.join(path_unite_subj, 'subj_in_groups_classification_quality_two_subjects_2_seconds.xlsx'), index=True)

In [ ]:
import random

In [ ]:
#ist_A, list_B = [], []
#all_indexes = index_mat + index_not_mat
#results_group = np.zeros((len(all_indexes),4))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
#index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
list_A, list_B = [], []
#mat = ['311','312','315','316','317','327','328','330','334']  
#not_mat = ['318','320','322','323','325','329','331','333']
#not_mat = ['320','322','323','325','329','331','333']
new_mat = random.sample(mat, 5)
new_not_mat = random.sample(not_mat, 5)
index_mat, index_not_mat = [indexes.index(i) for i in new_mat], [indexes.index(i) for i in new_not_mat] 
all_indexes = index_mat + index_not_mat
results_group = np.zeros((10,6))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
for count, s_ind in enumerate(all_indexes):
    list_A, list_B, list_C = [], [], []
    for ind in range(3):
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind]][i][ind],axis = 1)
                              for i in range(index_mat[index_mat!=s_ind].shape[0])]),axis=0)
        list_A.append(A)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind]][i][ind],axis = 1)
                              for i in range(index_not_mat[index_not_mat!=s_ind].shape[0])]),axis=0)
        list_B.append(B)
        C = np.stack(subj_list_features[s_ind][ind], axis = 1)
        # append new element to list
        list_C.append(C)
A, B, C = np.concatenate(list_A,axis=0), np.concatenate(list_B,axis=0), np.concatenate(list_C,axis=0)
x_train = np.concatenate((A,B),axis=0)
y_train = [0]*A.shape[0] + [1]*B.shape[0] 
x_train = x_train.reshape(x_train.shape[0],-1)
x_test = C.reshape(C.shape[0],-1)
if s_ind in index_mat:
    ll = 1
else:
    ll = 0               
y_test = [ll]*C.shape[0]
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
ac, auc = predict(x_train, x_test, y_train, y_test)
results_group[count, ind] = ac
results_group[count, ind+3] = auc

In [ ]:
y_test

In [ ]:
len(index_not_mat)

In [ ]:
mat

In [ ]:
#x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
scaler = StandardScaler()
scaler.fit(x_train)
scaler.transform(x_train)
scaler.transform(x_test)
model = LogisticRegressionCV(
   Cs=list(np.power(10.0, np.arange(-10, 10))),
   penalty='l2',
   scoring='roc_auc',
   random_state=0,
   max_iter=10000,
   fit_intercept=True,
   solver='newton-cg',
   tol=10 
                                                           ) 
model.fit(x_train, y_train)
Filter = model.coef_
inter = model.intercept_
X = x_train
inv_Y = 1.
X = X - X.mean(0, keepdims=True)
pattern = np.cov(X.T).dot(Filter.T.dot(1)).T
pattern = scaler.inverse_transform([pattern])[0]
results_group[count, ind] = predict_subject(x_test, y_test, pattern, inter)

In [ ]:
#ist_A, list_B = [], []
#all_indexes = index_mat + index_not_mat
#results_group = np.zeros((len(all_indexes),4))
#all_indexes_table = mat + not_mat
#all_indexes = np.array(all_indexes)
#index_mat, index_not_mat = np.array(index_mat), np.array(index_not_mat)
ll
for count, s_ind in enumerate(all_indexes):
    list_A, list_B, list_C = [], [], []
    for ind in range(3):
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat[index_mat!=s_ind]][i][ind],axis = 1)
                              for i in range(index_mat[index_mat!=s_ind].shape[0])]),axis=0)
        list_A.append(A)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat[index_not_mat!=s_ind]][i][ind],axis = 1)
                              for i in range(index_not_mat[index_not_mat!=s_ind].shape[0])]),axis=0)
        list_B.append(B)
        C = np.stack(subj_list_features[s_ind][ind], axis = 1)
        list_C.append(C)
    A, B, C = np.concatenate(list_A,axis=0), np.concatenate(list_B,axis=0), np.concatenate(list_C,axis=0)
    x_train = np.concatenate((A,B),axis=0)
    y_train = [0]*A.shape[0] + [1]*B.shape[0] 
    x_train = x_train.reshape(x_train.shape[0],-1)
    x_test = C.reshape(C.shape[0],-1)
    if s_ind in index_mat:
        ll = 1
    else:
        ll = 0               
y_test = [ll]*C.shape[0]
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.1)
results_group[count, ind] = predict(x_train, x_test, y_train, y_test)

In [ ]:
ll

In [ ]:
ll

In [ ]:
def predict_subject(X, y, Pattern, inter):
    #X = X - X.mean(0, keepdims=True)
    prec_x = np.linalg.pinv(np.cov(X.T))
    #Pattern = Pattern.ravel()
    new_filter = prec_x.dot(Pattern.T).T 
    scores = np.dot(X, new_filter.T) + inter
    scores.ravel()
    #scores_i = 1/(1+np.exp(scores))
    #y_predict = [1 if i > 0.2 else 0 for i in scores_i]
    indices = (scores > 0).astype(int) 
    cl = np.array([0,1])
    y_predict = cl[indices]
    a_s = accuracy_score(y, y_predict)
    return a_s

In [ ]:
# save results to table
datafr_groups = pd.DataFrame(results_all, columns=list_names_subj, index=['all_sibjects','mathematicians','not_mathematicians'])
datafr_groups.to_excel(os.path.join(path_unite_subj, 'subj_in_groups_classification_l2.xlsx'), index=True)

In [ ]:
# plot topomaps for subjects in groups
group_names = ['all_subjects','mathematicians','not_mathematicians']
for i, group_ind in enumerate(group_names):
    for j, key in enumerate(list(dict_cls.keys())):
        fig = plot_topo(coefs_all[i,j,...])
        fig.savefig(os.path.join(path_unite_topo, 'filters_patterns_{}_{}.png'
                                 .format(group_ind,['241_244','242_244','241_242'][j])), format='png', dpi=600)
        plt.close(fig)

In [ ]:
# classification mathematicians/non mathematicians
# create new coeffs minus gini
results_group, coefs_group = np.zeros((4,4)), np.zeros((4,2,len(fr_bands),len(chan1)))
list_A, list_B = [], []
for ind in range(4):
    if ind < 3:
        A =  np.concatenate(([np.stack([subj_list_features[j] for j in index_mat][i][ind],axis = 1)
                              for i in range(len(mat))]),axis=0)
        list_A.append(A)
        B =  np.concatenate(([np.stack([subj_list_features[j] for j in index_not_mat][i][ind],axis = 1)
                              for i in range(len(not_mat))]),axis=0)
        list_B.append(B)
    else:
        A, B = np.concatenate(list_A,axis=0), np.concatenate(list_B,axis=0)
    y = np.array(['0']*A.shape[0] + ['1']*B.shape[0])
    x = np.concatenate((A,B),axis=0)
    x_train, x_test, y_train, y_test = train_test_split(x.reshape(x.shape[0],-1),y,test_size=0.3)
    results_group[ind,...], model = predict(x_train, x_test, y_train, y_test)
    for name, i in zip(['patterns_', 'filters_'],[0,1]):
        coef = get_coef(model, name, inverse_transform=True)
        coefs_group[ind,i,...] = coef.reshape(len(fr_bands),-1)

In [ ]:
# save new results to table
datafr_math_not_math = pd.DataFrame(results_group, columns=['accuracy score','roc_auc score','sensitivity(TPR)',
                                             'specificity(TNR)'], index= ['241','242','244','all'])
datafr_math_not_math.to_excel(os.path.join(path_group, 'group_classification_l2.xlsx'), index=True)

In [ ]:
# plot and save topomaps for mat/not mat
for i in range(4):
    fig = plot_topo(coefs_group[i,...])
    fig.savefig(os.path.join(path_group_topo, 'filters_patterns_mat_not_mat_{}.png'
                             .format(['241','242','244','all'][i])), format='png', dpi=600)
    plt.close(fig)